In [ ]:
#1ST PRAC basic operations with TensorFlow 2 tensors Eager Execution

import tensorflow as tf

a = tf.constant([[1, 2], [3, 4]])
b = tf.constant([[5, 6], [7, 8]])

print("Add:\n", a + b)
print("MatMul:\n", a @ b)
print("ElemMul:\n", a * b)


In [ ]:
#2ND PRAC Preprocess and clean datasets for Generative AI

import pandas as pd
import numpy as np

data = {'Name': ['A', 'B', 'C', None],
        'Age': [25, np.nan, 22, 28],
        'City': ['Pune', 'Mumbai', 'Delhi', 'Pune']}
df = pd.DataFrame(data)

# Handle missing data (updated method)
df['Name'] = df['Name'].fillna('Unknown')
df['Age'] = df['Age'].fillna(df['Age'].mean())

# Normalize numerical column
df['Age'] = (df['Age'] - df['Age'].min()) / (df['Age'].max() - df['Age'].min())

# Encode categorical variable
df = pd.get_dummies(df, columns=['City'])
print(df)


In [ ]:
#3RD PRAC Use Matplotlib or Seaborn to visualize data

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load dataset
df = sns.load_dataset('iris')

# 1️⃣ Histogram
sns.histplot(df['sepal_length'], kde=True)
plt.title("Sepal Length Distribution")
plt.show()

# 2️⃣ Scatter plot
sns.scatterplot(x='sepal_length', y='petal_length', hue='species', data=df)
plt.title("Sepal vs Petal Length")
plt.show()

# 3️⃣ Correlation heatmap (numeric columns only)
numeric_df = df.select_dtypes(include=['float', 'int'])  # Filter numeric columns
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap (Numeric Features)")
plt.show()


In [ ]:
#PRAC 4 Implement word embeddings (e.g., Word2Vec or GloVe)
!pip install faiss-cpu
import gensim.downloader as api, faiss, numpy as np

m = api.load('glove-wiki-gigaword-50')
vecs = np.array([m[w] for w in m.index_to_key[:5000]], 'float32')
idx = faiss.IndexFlatL2(50); idx.add(vecs)

word = "car"
D, I = idx.search(np.array([m[word]], 'float32'), 5)
print(f"Words semantically similar to '{word}':", [m.index_to_key[i] for i in I[0]])


In [ ]:
#PRAC 5 pre-trained diffusion model (e.g., Denoising Diffusion Probabilistic

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Load and preprocess MNIST
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.0
x_train = np.expand_dims(x_train, -1).astype("float32")

# Create a simple random generator (for demo only)
noise = tf.random.normal((16, 28, 28, 1))
generated = noise  # since we don’t have pretrained model

# Display random "generated" images
plt.figure(figsize=(6,6))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(generated[i,:,:,0], cmap='gray')
    plt.axis('off')
plt.show()

In [ ]:
#PRAC 6 CLIP-guided diffusion model to perform text-to-image generation.


# Simple Text-to-Image using Pollinations API + CLIP similarity
# Fast, lightweight, and shows image inline

!pip install -q git+https://github.com/openai/CLIP.git torchvision pillow requests

import torch, requests
from PIL import Image
from io import BytesIO
import clip
import matplotlib.pyplot as plt

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Text prompt
prompt = "A futuristic cityscape with flying cars"

# Generate image quickly using Pollinations API
url = f"https://image.pollinations.ai/prompt/{requests.utils.quote(prompt)}"
image = Image.open(BytesIO(requests.get(url).content)).convert("RGB")

# Show image inline
plt.imshow(image)
plt.axis("off")
plt.title(prompt)
plt.show()

# Evaluate image-text similarity using CLIP
text = clip.tokenize([prompt]).to(device)
img = preprocess(image).unsqueeze(0).to(device)
with torch.no_grad():
    score = (model.encode_image(img) @ model.encode_text(text).T).item()

print(f"CLIP Similarity Score: {score:.4f}")


In [ ]:
#PRAC 7 (e.g., DistilGPT-2) using LoRA on a text corpus

# Single-cell: LoRA fine-tune (DistilGPT-2) -> generate -> perplexity
# If libs missing, it will install a compatible set (may take a minute once).
try:
    import torch, transformers, peft
except Exception:
    !pip install -q torch transformers==4.37.2 peft==0.7.1 datasets accelerate==0.25.0
    import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from torch.utils.data import Dataset, DataLoader
import math

# ----- tiny dataset (for quick demo) -----
texts = [
  "AI is transforming technology.",
  "LoRA fine-tuning saves memory.",
  "Transformers are great for text generation.",
  "AI will shape the future."
] * 40   # repeat to give the model some signal

# ----- tokenizer / model / LoRA -----
tok = AutoTokenizer.from_pretrained("distilgpt2")
tok.pad_token = tok.eos_token
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
lora_cfg = LoraConfig(r=8, lora_alpha=16, target_modules=["c_attn"], task_type="CAUSAL_LM")
model = get_peft_model(model, lora_cfg)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ----- tiny Dataset -----
class T(Dataset):
    def __init__(self, texts, tok):
        enc = tok(texts, padding="max_length", truncation=True, max_length=64, return_tensors="pt")
        self.ids, self.att = enc["input_ids"], enc["attention_mask"]
    def __len__(self): return self.ids.size(0)
    def __getitem__(self, i): return {"input_ids": self.ids[i], "attention_mask": self.att[i]}

ds = T(texts, tok)
loader = DataLoader(ds, batch_size=4, shuffle=True)

# ----- quick training loop (1 epoch, very short) -----
opt = torch.optim.AdamW(model.parameters(), lr=5e-5)
model.train()
for epoch in range(1):
    for batch in loader:
        ids = batch["input_ids"].to(device)
        att = batch["attention_mask"].to(device)
        out = model(input_ids=ids, attention_mask=att, labels=ids)
        loss = out.loss
        loss.backward()
        opt.step(); opt.zero_grad()
    print(f"Epoch {epoch+1} loss: {loss.item():.4f}")

# ----- generate a sample -----
model.eval()
prompt = "Artificial intelligence will"
ids = tok(prompt, return_tensors="pt").input_ids.to(device)
gen = model.generate(ids, max_length=50, do_sample=True, top_p=0.9, num_return_sequences=1)
print("\n=== GENERATED ===\n", tok.decode(gen[0], skip_special_tokens=True))

# ----- approximate perplexity on few training samples -----
model.eval()
losses = []
with torch.no_grad():
    for i in range(8):
        item = ds[i]
        ids = item["input_ids"].unsqueeze(0).to(device)
        out = model(input_ids=ids, labels=ids)
        losses.append(out.loss.item())
ppl = math.exp(sum(losses)/len(losses))
print(f"\nApprox Perplexity: {ppl:.2f}")


In [ ]:
#PRAC 8 Evaluate outputs using BLEU or ROUGE metrics.
!pip install evaluate
from transformers import pipeline
import evaluate
!pip install rouge_score

summarizer = pipeline("summarization", model="t5-small")
text = "Artificial Intelligence enables machines to learn from data and perform tasks that normally need human intelligence."
summary = summarizer(text, max_length=40, min_length=10, do_sample=False)[0]['summary_text']

qa = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
ans = qa(question="Who developed GPT models?", context="GPT models are large language models developed by OpenAI.")['answer']

rouge = evaluate.load("rouge")
score = rouge.compute(predictions=[summary], references=["AI helps machines perform human-like tasks."])

print("Summary:", summary)
print("Answer:", ans)
print("ROUGE Score:", score["rouge1"])


In [ ]:
# PRAC 9: Retrieval-Augmented Generation (RAG)
!pip install -q faiss-cpu sentence-transformers transformers

from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss, numpy as np

# 1) Small document collection
docs = [
  "Paris is the capital of France and is known for the Eiffel Tower.",
  "The Nile is the longest river in Africa and flows through Egypt.",
  "Python is a popular programming language for data science and AI.",
  "The mitochondrion is the powerhouse of the cell.",
  "Cricket is a bat-and-ball game popular in India, England, and Australia."
]

# 2) Create embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embs = embedder.encode(docs, convert_to_numpy=True).astype("float32")

# 3) Build FAISS index
index = faiss.IndexFlatL2(doc_embs.shape[1])
index.add(doc_embs)

# 4) Ask question
question = "Which city has the Eiffel Tower?"

# 5) Retrieve top-2 relevant documents
q_emb = embedder.encode([question], convert_to_numpy=True).astype("float32")
D, I = index.search(q_emb, k=2)
retrieved_docs = [docs[i] for i in I[0]]

# 6) Combine retrieved context
context = " ".join(retrieved_docs)

# 7) Use QA model for final answer
qa = pipeline("question-answering", model="deepset/roberta-base-squad2")
answer = qa(question=question, context=context)

print("Retrieved Documents:")
for r in retrieved_docs:
    print("-", r)

print("\nQuestion:", question)
print("Answer:", answer['answer'])


In [ ]:
#PRAC 10 Apply model optimization techniques such as pruning or quantization

# Quantize DistilBERT and test inference speed
!pip install -q transformers torch

import torch, time
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load model and tokenizer
tok = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Sample input
text = "Artificial intelligence is transforming the world."
inputs = tok(text, return_tensors="pt")

# Original model speed
start = time.time()
_ = model(**inputs)
orig_time = time.time() - start

# Quantize model (8-bit)
quant_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

# Quantized model speed
start = time.time()
_ = quant_model(**inputs)
quant_time = time.time() - start

# Compare
print(f"Original time: {orig_time:.4f}s | Quantized time: {quant_time:.4f}s")
print(f"Memory reduced from {model.num_parameters()/1e6:.1f}M → {quant_model.num_parameters()/1e6:.1f}M parameters")


In [ ]:
# PRAC 11: Bias in the outputs of a pre-trained generative model
!pip install -q transformers

from transformers import pipeline

# Load GPT-2 model
gen = pipeline("text-generation", model="gpt2", pad_token_id=50256)

# Prompts that might show bias
prompts = ["The nurse said that", "The engineer said that"]

print("=== Raw Model Outputs (Potential Bias) ===")
for p in prompts:
    out = gen(
        p,
        max_length=25,          # slightly longer for smoother completion
        num_return_sequences=1,
        do_sample=True,
        top_p=0.9,              # nucleus sampling to reduce repetition
        temperature=0.7,         # more controlled generation
    )[0]['generated_text']
    print(f"\nPrompt: {p}\nOutput: {out.strip()}")

# --- Simple bias mitigation: remove gendered words ---
def mitigate_bias(text):
    gendered_words = {"he", "she", "him", "her", "his", "hers"}
    clean = " ".join([w for w in text.split() if w.lower() not in gendered_words])
    return clean

print("\n=== After Simple Bias Mitigation ===")
for p in prompts:
    out = gen(
        p,
        max_length=25,
        num_return_sequences=1,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
    )[0]['generated_text']
    mitigated = mitigate_bias(out.strip())
    print(f"\nPrompt: {p}\nOutput: {mitigated}")


In [ ]:
#PRAC 12 simple autonomous AI agent using Python for a text-based task

from transformers import pipeline
import time

agent = pipeline("text-generation", model="distilgpt2")

def ai_agent(name, message):
    response = agent(message, max_length=50, num_return_sequences=1)[0]['generated_text']
    print(f"\n{name}: {response}\n")
    return response

msg = "Hello! What do you think about AI?"
for i in range(3):
    msg = ai_agent("Agent 1", msg)
    time.sleep(1)
    msg = ai_agent("Agent 2", msg)
    time.sleep(1)


In [ ]:
from transformers import pipeline

agent = pipeline("text-generation", model="gpt2")

def ai_agent(name, message):
    response = agent(message, max_length=40, truncation=True)[0]['generated_text']
    print(f"{name}: {response}\n")
    return response

msg = "Hello! What do you think about AI?"
for i in range(1):
    msg = ai_agent("Agent 1", msg)
    msg = ai_agent("Agent 2",msg)